# Experimenting and EDA for Dataset

In [1]:
import pymysql
import pandas as pd
from dotenv import dotenv_values
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px



In [2]:
# Getting env var
secrets = dotenv_values()

host =secrets['DB_HOST']
name = secrets['DB_NAME']
user = secrets['DB_USER']
pwd = secrets['DB_PWD']


In [3]:
# Connecting to DB

connection = pymysql.connect(
    host = host,
    user = user,
    password = pwd,
    database = name
    )

cursor = connection.cursor()
cursor.execute("SELECT * FROM churn_status")

results = cursor.fetchall()

print(len(results))

cursor.close()
connection.close()

7043


# EDA
Done from local data files

---

To Do List
- Join unscaled and unprocessed datasets. 
- Remove rows without churn label (if any)
- Check for class imbalance -> if have, perform stratified split 
- Split into train-validation (70-30)
- Perform outlier removal only on train set
- After outlier removal, preprocess (categorical encode and scale) columns in both datasets
- Perform PCA to determine which features to use
- Train machine learning model with kfold cross validation

In [4]:
# File paths
ACC_PATH = "data_given/1_account.csv"
ACC_USAGE_PATH = "data_given/2_account_usage.csv"
CHURN_STATUS_PATH = "data_given/3_churn_status.csv"
CITY_PATH = "data_given/4_city.csv"
CUSTOMER_PATH = "data_given/5_customer.csv"

### Joining all unprocessed tables

In [29]:
# Loading DFs
CUSTOMER_DF = pd.read_csv(CUSTOMER_PATH)
ACCOUNT_DF = pd.read_csv(ACC_PATH)
ACC_USAGE_DF = pd.read_csv(ACC_USAGE_PATH)
CHURN_STATUS_DF = pd.read_csv(CHURN_STATUS_PATH)
CITY_DF = pd.read_csv(CITY_PATH)

# Joining customer and account tables on customer_id
DF = pd.merge(CUSTOMER_DF, ACCOUNT_DF, on='customer_id', how='inner')
# Joining df with account_usage_df on account_id
DF = pd.merge(DF, ACC_USAGE_DF, on='account_id', how='inner')
# Joining df with churn_status_df on customer_id
DF = pd.merge(DF, CHURN_STATUS_DF, on='customer_id', how='inner')
# Joining df with city_df on area_id
DF = pd.merge(DF, CITY_DF, on='zip_code', how='inner')

In [30]:
DF.head()

,customer_id,gender,age,senior_citizen,married,num_dependents,zip_code,account_id,tenure_months,num_referrals,...,total_refunds,status,churn_label,churn_category,churn_reason,area_id,city,latitutde,longitude,population
0,0002-ORFBO,Female,37,No,Yes,0,93225,XSWV-PAYXZ,9,2,...,0.00,Stayed,No,NaN,NaN,649,Frazier Park,34.827662,-118.999073,4498
1,5183-SNMJQ,Male,32,No,No,0,93225,DZQJ-ZMREB,10,0,...,43.25,Stayed,No,NaN,NaN,649,Frazier Park,34.827662,-118.999073,4498
2,6847-KJLTS,Female,72,Yes,Yes,0,93225,VLBU-IQLTI,58,8,...,0.00,Stayed,No,NaN,NaN,649,Frazier Park,34.827662,-118.999073,4498
3,8788-DOXSU,Male,46,No,No,0,93225,HSKL-QCEUU,59,0,...,0.00,Stayed,No,NaN,NaN,649,Frazier Park,34.827662,-118.999073,4498
4,0003-MKNFE,Male,46,No,No,0,91206,VFUN-NFDPJ,9,0,...,38.33,Stayed,No,NaN,NaN,184,Glendale,34.162515,-118.203869,31297


In [ ]:
def convert_to_categorical(df:pd.DataFrame)->pd.DataFrame:
    """
    Converts the following columns to categorical data
    
    Yes No to 1/0
    - has_internet_service
    - has_phone_service
    - has_unlimited_data
    - has_multiple_lines
    - has_premium_tech_support
    - has_online_security
    - has_online_backup
    - has_device_protection
    - paperless_billing
    - stream_movie
    - stream_music
    - stream_tv
    - churn_label
    - senior_citizen
    - married
    - gender (1 for male, 0 for female)
    

    """
    # Creating output_df
    output_df = df.copy()

     # Converting yes/no to 1/0
    mapping = {'Yes':1, 'No':0}
    output_df['has_internet_service'] = df['has_internet_service'].map(mapping)
    output_df['has_phone_service'] = df['has_phone_service'].map(mapping)
    output_df['has_unlimited_data'] = df['has_unlimited_data'].map(mapping)
    output_df['has_multiple_lines'] = df['has_multiple_lines'].map(mapping)
    output_df['has_premium_tech_support'] = df['has_premium_tech_support'].map(mapping)
    output_df['has_online_security'] = df['has_online_security'].map(mapping)
    output_df['has_online_backup'] = df['has_online_backup'].map(mapping)
    output_df['has_device_protection'] = df['has_device_protection'].map(mapping)
    output_df['paperless_billing'] = df['paperless_billing'].map(mapping)
    output_df['stream_movie'] = df['stream_movie'].map(mapping)
    output_df['stream_music'] = df['stream_music'].map(mapping)
    output_df['stream_tv'] = df['stream_tv'].map(mapping)
    output_df['senior_citizen'] = df['senior_citizen'].map(mapping)
    output_df['married'] = df['married'].map(mapping)
    output_df['gender'] = df['gender'].map(mapping)
    # Some are missing churn_labels, but have status as "churned"
    output_df['churn_label'] = df['churn_label'].fillna(0)
    output_df.loc[df['status'] == 'Churned', 'churn_label'] = 1
    mapping = {'Yes':1, 'No':0, }
    output_df['churn_label'] = df['churn_label'].map(mapping)



# Outlier Analysis

In [9]:
def detect_outliers_zscore(df:pd.DataFrame, threshold:int=3)->pd.DataFrame:
    """
    Detect outliers using the z-score method.
    Returns a boolean mask indicating the outliers.
    """
    z_scores = (df - np.mean(df)) / np.std(df)
    outliers = np.abs(z_scores) > threshold
    return outliers

def plot_boxplot(df_column:pd.Series)->None:
    """
    Plots a box plot based on the given DataFrame column using Plotly.
    """
    # Plotting boxplot using Plotly
    fig = px.box(df_column, title='Box Plot')

    # Show the plot
    fig.show()

def get_value_at_percentile(df:pd.DataFrame, column:str, percentile:int)->float:
    """
    Returns the value at a specific percentile in a DataFrame column.
    """
    # Get the values from the specified column
    column_values = df[column].values

    # Calculate the percentile value
    value_at_percentile = np.percentile(column_values, percentile)

    return value_at_percentile

def remove_rows_by_values(df:pd.DataFrame, column:str, lower_bound:float, upper_bound:float):
    """
    Removes rows from a DataFrame based on specific column values.
    """
    # Create a boolean mask to filter rows with the specified column values
    mask = (df[column] >= lower_bound) & (df[column] <= upper_bound)

    # Filter the DataFrame using the boolean mask
    updated_df = df[mask]

    return updated_df

# Outlier Analysis Summary
### ACC_DF

1. tenure_months

In the dataset, 362 rows have tenure_months at 72 while 0 rows have tenure_months at 71. As 72 months is 3 years, this suggests that the maximum contract period is 3 years. 362 rows consists of approximately 5% of the entire data set we have currently, hence I will not be removing any outliers based on tenure_months.

2. num_referrals

I decide to keep the 0th to 95th percentile of the rows based on the number of referrals. This meant the cutoff value would be 9, and 225 rows will be removed as a results.


In [ ]:
# ACC_DF = pd.read_csv(ACC_PATH)
# ACC_DF.head()
# # Keeping from 80th percentile to 0th percentile for tenure_months
# # plot_boxplot(ACC_DF['num_referrals'])
# get_value_at_percentile(ACC_DF, 'num_referrals', 80)

# def acc_df_remove_outliers(df:pd.DataFrame):
#     # Tenure_months: Keep from 0th to 80th percentile
#     # Num_referrals: Keep from 0th to 80th percentile
#     tenure_months_upper_bound = get_value_at_percentile(df, 'tenure_months', 90)
#     num_referarals_upper_bound = get_value_at_percentile(df, 'num_referrals', 80)

#     # Removing outliers
#     output_df = remove_rows_by_values(df, 'tenure_months', 0, tenure_months_upper_bound)
#     print(f"Removed {len(df) - len(output_df)} rows from the DataFrame based on the 'tenure_months' column values.")

#     output_df = remove_rows_by_values(df, 'num_referrals', 0, num_referarals_upper_bound)
#     print(f"Removed {len(df) - len(output_df)} rows from the DataFrame based on the 'num_referrals' column values.")

#     return output_df

# plot_boxplot(ACC_DF['tenure_months'])
# cleaned_acc_df = acc_df_remove_outliers(ACC_DF)
# plot_boxplot(cleaned_acc_df['tenure_months'])




### General Helper Functions

In [ ]:
def get_unique_values(df:pd.DataFrame)->dict:
    """
    Runs through the columns of a dataframe and prints the unique values of each column. 
    """
    dict_unique_values = {}
    for cols in df.columns:
        dict_unique_values[cols] = df[cols].unique()
    return dict_unique_values

In [ ]:
def get_NaN_count(df:pd.DataFrame)->dict:
    """
    Returns the number of NaN values for each column in a dictionary.
    """
    nan_count = df.isna().sum().to_dict()
    return nan_count

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

def preprocess_acc_df(df:pd.DataFrame)->pd.DataFrame:
    """
    Preprocesses the account dataframe as below:

    Converts follow columns (yes/no) to 1/0
    - has_internet_service
    - has_phone_service
    - has_unlimited_data
    - has_multiple_lines
    - has_premium_tech_support
    - has_online_security
    - has_online_backup
    - has_device_protection
    - paperless_billing

    Converts follow columns to ordinal values:
    - contract_type (one-year, month-to-month, two-year) => (1,0,2) 

    One-hot encodes following columns:
    - payment_method
    - internet_type

    Scales following columns:
    - tenure_months
    """
    # Creating a new df
    output_df = pd.DataFrame()

    # Copying over account_id	customer_id	tenure_months
    output_df['account_id'] = df['account_id']
    output_df['customer_id'] = df['customer_id']

    # Converting yes/no to 1/0
    mapping = {'Yes':1, 'No':0}

    output_df['has_internet_service'] = df['has_internet_service'].map(mapping)
    output_df['has_phone_service'] = df['has_phone_service'].map(mapping)
    output_df['has_unlimited_data'] = df['has_unlimited_data'].map(mapping)
    output_df['has_multiple_lines'] = df['has_multiple_lines'].map(mapping)
    output_df['has_premium_tech_support'] = df['has_premium_tech_support'].map(mapping)
    output_df['has_online_security'] = df['has_online_security'].map(mapping)
    output_df['has_online_backup'] = df['has_online_backup'].map(mapping)
    output_df['has_device_protection'] = df['has_device_protection'].map(mapping)
    output_df['paperless_billing'] = df['paperless_billing'].map(mapping)

    mapping = {'Month-to-Month':0, 'One Year':1, 'Two Year':2}
    output_df['contract_type'] = df['contract_type'].map(mapping)

    # One-hot encoding
    # one_hot_encoder = OneHotEncoder()
    # one_hot_encoded_cols = ["payment_method", "internet_type"]
    # encoded_df = pd.DataFrame(one_hot_encoder.fit_transform(df[one_hot_encoded_cols]))
    # encoded_df.columns = one_hot_encoder.get_feature_names_out(one_hot_encoded_cols)
    # print(encoded_df.head())

    # One-hot encoding
    one_hot_encoder = OneHotEncoder()
    one_hot_encoded_cols = ["payment_method", "internet_type"]
    encoded_features = one_hot_encoder.fit_transform(df[one_hot_encoded_cols])
    encoded_df = pd.DataFrame(encoded_features.toarray(), columns=one_hot_encoder.get_feature_names_out(one_hot_encoded_cols))
    output_df = pd.concat([output_df, encoded_df], axis=1)

    # Scaling
    scaler = MinMaxScaler()
    output_df['tenure_months'] = scaler.fit_transform(df[['tenure_months']])

    return output_df

### Account Usage 

In [ ]:
def preprocess_acc_usage_df(df:pd.DataFrame)->pd.DataFrame:
    """
    Preprocess the account usage dataframe as below:
    Scale the following columns:
    - avg_long_distance_fee_monthly
    - total_long_distance_fee
    - avg_gb_download_monthly
    - total_monthly_fee
    - total_chargers_quarter
    - total_refunds
    Converts following col to 1/0:
    - stream_move
    - stream_music
    - stream_tv
    """
    # Create new df
    output_df = pd.DataFrame()

    # Copying over acc_id
    output_df['account_id'] = df['account_id']

    # Converting yes/no to 1/0
    mapping = {'Yes':1, 'No':0}

    output_df['stream_movie'] = df['stream_movie'].map(mapping)
    output_df['stream_music'] = df['stream_music'].map(mapping)
    output_df['stream_tv'] = df['stream_tv'].map(mapping)

    # Scaling
    scaler = MinMaxScaler()
    output_df['avg_long_distance_fee_monthly'] = scaler.fit_transform(df[['avg_long_distance_fee_monthly']])
    output_df['total_long_distance_fee'] = scaler.fit_transform(df[['total_long_distance_fee']])
    output_df['avg_gb_download_monthly'] = scaler.fit_transform(df[['avg_gb_download_monthly']])
    output_df['total_monthly_fee'] = scaler.fit_transform(df[['total_monthly_fee']])
    output_df['total_charges_quarter'] = scaler.fit_transform(df[['total_charges_quarter']])
    output_df['total_refunds'] = scaler.fit_transform(df[['total_refunds']])

    return output_df

### Churn Status

In [ ]:
from category_encoders import BinaryEncoder
from sklearn.preprocessing import LabelEncoder

def preprocess_churn_df(df:pd.DataFrame)->pd.DataFrame:
    """
    Preprocess the churn status dataframe as below:
    Convert following col to 1/0:
    - churn_label

    Binary encodes:
    - churn_category
    
    Label-encodes:
    - status

    Drops the following:
    - churn_reason -> Not planning to do NLP
    """
    # Create new df
    output_df = pd.DataFrame()

    # Copying over acc_id
    output_df['customer_id'] = df['customer_id']

    # Converting yes/no to 1/0
    # Additional step to check status 
    output_df['churn_label'] = df['churn_label'].fillna(0)
    output_df.loc[df['status'] == 'Churned', 'churn_label'] = 1
    mapping = {'Yes':1, 'No':0, }
    output_df['churn_label'] = df['churn_label'].map(mapping)

    # Binary Encoding
    binary_encoder = BinaryEncoder(cols=['churn_category'])
    binary_encoder.fit_transform(df['churn_category'])
    churn_cat = binary_encoder.transform(df['churn_category'])
    output_df = pd.concat([output_df, churn_cat], axis=1)

    # Label Encoding
    label_encoder = LabelEncoder()
    output_df['status'] = label_encoder.fit_transform(df['status'])

    return output_df

### City DF

In [ ]:
def preprocess_city_df(df:pd.DataFrame)->pd.DataFrame:
    """
    Preprocess the city dataframe as below:
    Min-max scales population column
    Keep area_id and population only
    """
    # Creating new df
    output_df = pd.DataFrame()

    # Copying over area_id
    output_df['zip_code'] = df['zip_code']

    # Scaling
    scaler = MinMaxScaler()
    output_df['population'] = scaler.fit_transform(df[['population']])

    return output_df

### Customer DF

In [ ]:
def preprocess_customer_df(df:pd.DataFrame)->pd.DataFrame:
    """
    Preprocess the customer dataframe as below:
    Convert following col to 1/0:
    - senior_citizen
    - married
    - gender (1 for male, 0 for female)
    """
    # Create new df
    output_df = pd.DataFrame()

    # Copying over customer_id, zip_code
    output_df['customer_id'] = df['customer_id']
    output_df['zip_code'] = df['zip_code']

    # Converting columns to 1/0
    mapping = {'Yes':1, 'No':0, "Male":1, "Female":0}
    output_df['senior_citizen'] = df['senior_citizen'].map(mapping)
    output_df['married'] = df['married'].map(mapping)
    output_df['gender'] = df['gender'].map(mapping)

    return output_df

### Preprocessing all CSV

In [ ]:
# Loading dfs from CSV
ACCOUNT_DF = pd.read_csv(ACC_PATH)
ACCOUNT_USAGE_DF = pd.read_csv(ACC_USAGE_PATH)
CHURN_STATUS_DF = pd.read_csv(CHURN_STATUS_PATH)
CITY_DF = pd.read_csv(CITY_PATH)
CUSTOMER_DF = pd.read_csv(CUSTOMER_PATH)

# Preprocessing dfs
account_df = preprocess_acc_df(ACCOUNT_DF)
account_usage_df = preprocess_acc_usage_df(ACCOUNT_USAGE_DF)
churn_status_df = preprocess_churn_df(CHURN_STATUS_DF)
city_df = preprocess_city_df(CITY_DF)
customer_df = preprocess_customer_df(CUSTOMER_DF)

## Joining Tables

In [ ]:
# Joining customer and account tables on customer_id
df = pd.merge(customer_df, account_df, on='customer_id', how='inner')
# Joining df with account_usage_df on account_id
df = pd.merge(df, account_usage_df, on='account_id', how='inner')
# Joining df with churn_status_df on customer_id
df = pd.merge(df, churn_status_df, on='customer_id', how='inner')
# Joining df with city_df on area_id
df = pd.merge(df, city_df, on='zip_code', how='inner')